In [10]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import random
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  # 추가

RANDOM_STATE = 42

np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

### main code

In [11]:
bert_word_embed = pd.read_csv('/Users/kookbab/Desktop/연구 정리 KCI/코드/04.making_graphs/numerical_values/BERT_word_embed.csv')
bert_word_topic_sim = pd.read_csv('/Users/kookbab/Desktop/연구 정리 KCI/코드/04.making_graphs/numerical_values/BERT_word_topic_sim.csv')
lda_word_topic_prob = pd.read_csv('/Users/kookbab/Desktop/연구 정리 KCI/코드/04.making_graphs/numerical_values/LDA_word_topic_prob.csv')

In [12]:
bert_word_embed = bert_word_embed.rename(columns={'Unnamed: 0': 'word'})
bert_word_topic_sim = bert_word_topic_sim.rename(columns={'Unnamed: 0': 'word'})
lda_word_topic_prob = lda_word_topic_prob.rename(columns={'Unnamed: 0': 'word'})

In [13]:
# from sklearn.preprocessing import StandardScaler
# bert_word_embed = pd.concat([bert_word_embed.iloc[:,0], pd.DataFrame(StandardScaler().fit_transform(bert_word_embed.iloc[:,1:]))], axis=1)
# bert_word_topic_sim = pd.concat([bert_word_topic_sim.iloc[:,0], pd.DataFrame(StandardScaler().fit_transform(bert_word_topic_sim.iloc[:,1:]))], axis=1)
# lda_word_topic_prob = pd.concat([lda_word_topic_prob.iloc[:,0], pd.DataFrame(StandardScaler().fit_transform(lda_word_topic_prob.iloc[:,1:]))], axis=1)

In [14]:
bert_word_embed.head()

,word,0,1,2,3,4,5,6,7,8,...,374,375,376,377,378,379,380,381,382,383
0,가격,0.064543,0.152627,-0.028713,0.138644,-0.113938,-0.528496,0.169260,0.340302,0.535322,...,0.026984,-0.102562,-0.210618,-0.214387,-0.320685,0.193843,0.283838,-0.522911,-0.423793,-0.019472
1,가공,-0.008603,0.121856,-0.047261,0.081057,-0.142101,-0.108456,0.335849,0.114816,0.027490,...,0.002784,-0.064741,0.002136,-0.090872,-0.104440,0.110108,0.163750,-0.131098,0.044682,0.079863
2,가구,0.478547,-0.196509,0.027212,0.362131,0.290815,0.285264,-0.390044,0.298700,0.223424,...,-0.511272,-0.169213,0.098758,0.265926,-0.175584,0.115909,0.242987,0.189564,-0.148971,0.350775
3,가늠,0.057586,0.108453,-0.035356,0.043716,-0.097760,-0.092718,0.432894,0.132148,0.023564,...,0.025807,-0.061676,0.002965,-0.065089,-0.215861,0.176212,0.236612,-0.084089,-0.023032,0.063043
4,가능,0.165264,-0.176418,0.033578,-0.047770,-0.105405,-0.152465,0.076640,-0.003154,-0.160596,...,0.032628,-0.011553,0.118655,0.065454,-0.307813,0.211941,0.296655,-0.104277,0.132616,-0.060043


In [15]:
bert_word_topic_sim.head()

,word,BERT_Topic_0,BERT_Topic_1,BERT_Topic_2,BERT_Topic_3,BERT_Topic_4,BERT_Topic_5,BERT_Topic_6,BERT_Topic_7,BERT_Topic_8,BERT_Topic_9,BERT_Topic_10,BERT_Topic_11,BERT_Topic_12,BERT_Topic_13,BERT_Topic_14,BERT_Topic_15,BERT_Topic_16
0,가격,0.141422,0.155586,0.080380,0.150533,0.147157,0.177212,0.231533,0.113432,0.118626,0.049367,0.092995,0.099968,0.083575,0.037997,0.085731,0.109112,0.049306
1,가공,0.433861,0.419134,0.346928,0.497120,0.353951,0.347922,0.301154,0.445872,0.425080,0.363015,0.312770,0.329950,0.378247,0.320899,0.302277,0.390822,0.259295
2,가구,0.087519,0.072110,0.077759,0.058951,0.151163,0.125775,0.001572,0.057238,0.050947,0.051677,0.112088,0.090368,0.041744,0.088561,0.028610,0.065818,0.074075
3,가늠,0.443324,0.422610,0.362140,0.523029,0.381890,0.312519,0.300304,0.454108,0.438924,0.386340,0.314339,0.322147,0.399118,0.315127,0.354469,0.403826,0.265403
4,가능,0.368289,0.365992,0.289751,0.373412,0.343575,0.332016,0.273542,0.385838,0.360268,0.277694,0.312099,0.274423,0.351035,0.247753,0.322932,0.347216,0.231434


In [16]:
lda_word_topic_prob.head()

,word,LDA_Topic_0,LDA_Topic_1,LDA_Topic_2,LDA_Topic_3,LDA_Topic_4,LDA_Topic_5,LDA_Topic_6,LDA_Topic_7,LDA_Topic_8,...,LDA_Topic_17,LDA_Topic_18,LDA_Topic_19,LDA_Topic_20,LDA_Topic_21,LDA_Topic_22,LDA_Topic_23,LDA_Topic_24,LDA_Topic_25,LDA_Topic_26
0,개개인,0.000006,0.000004,0.000010,0.000066,0.000022,0.000110,0.000007,0.000005,0.001123,...,0.000213,0.000007,0.000013,0.000574,0.000175,0.000010,0.000026,0.000526,0.000070,0.000039
1,개정,0.000005,0.000003,0.000010,0.000009,0.000033,0.001174,0.000008,0.000005,0.001112,...,0.000007,0.000007,0.000014,0.000997,0.000288,0.000583,0.000018,0.000458,0.000013,0.000038
2,검토,0.000921,0.000233,0.000014,0.000109,0.001256,0.000888,0.000328,0.001066,0.003352,...,0.000395,0.000026,0.000049,0.000373,0.000667,0.000012,0.000019,0.000465,0.000628,0.000045
3,경험,0.000937,0.001249,0.003654,0.002961,0.002959,0.004043,0.002193,0.002497,0.000253,...,0.001901,0.009241,0.005576,0.002300,0.001062,0.003913,0.004051,0.004451,0.003614,0.000095
4,계획,0.001014,0.000992,0.012323,0.006993,0.002540,0.006171,0.000390,0.006434,0.002632,...,0.001649,0.001929,0.013027,0.003542,0.007001,0.003849,0.005627,0.001727,0.012552,0.000117


# Calculate Cosine Similarity

In [17]:
data1 = pd.concat([bert_word_embed[['word']], pd.DataFrame(cosine_similarity(bert_word_embed.drop(columns='word').values))], axis=1)
# data1.to_parquet('/Users/kookbab/Desktop/연구 정리 KCI/코드/04.making_graphs/numerical_values/COS_bert_word_embed.parquet', engine="pyarrow")

data2 = pd.concat([bert_word_topic_sim[['word']], pd.DataFrame(cosine_similarity(bert_word_topic_sim.drop(columns='word').values))], axis=1)
# data2.to_parquet('/Users/kookbab/Desktop/연구 정리 KCI/코드/04.making_graphs/numerical_values/COS_bert_word_topic_sim.parquet', engine="pyarrow")

data3 = pd.concat([lda_word_topic_prob[['word']], pd.DataFrame(cosine_similarity(lda_word_topic_prob.drop(columns='word').values))], axis=1)
# data3.to_parquet('/Users/kookbab/Desktop/연구 정리 KCI/코드/04.making_graphs/numerical_values/COS_lda_word_topic_prob.parquet', engine="pyarrow")

,0,1,2,3,4,5,6,7,8,9,...,2587,2588,2589,2590,2591,2592,2593,2594,2595,2596
0,1.000000,0.407323,0.175059,0.386242,0.314844,0.359547,0.501066,0.409526,0.344834,0.417297,...,0.364112,0.367821,0.176155,0.243669,0.310015,0.380453,0.260374,0.384227,0.277409,0.390131
1,0.407323,1.000000,0.351145,0.936133,0.703885,0.626613,0.905683,0.938638,0.880317,0.898000,...,0.771746,0.881804,0.522162,0.430400,0.798155,0.667386,0.734768,0.824508,0.553637,0.895474
2,0.175059,0.351145,1.000000,0.332309,0.286551,0.245327,0.283685,0.336133,0.261759,0.413682,...,0.285084,0.284752,0.113728,0.155702,0.188021,0.195877,0.221872,0.314224,0.205434,0.333461
3,0.386242,0.936133,0.332309,1.000000,0.742892,0.677141,0.930200,0.961065,0.938620,0.910219,...,0.827239,0.940204,0.488092,0.452694,0.836859,0.647204,0.757787,0.857116,0.650407,0.923010
4,0.314844,0.703885,0.286551,0.742892,1.000000,0.874303,0.718404,0.739160,0.700874,0.794549,...,0.680254,0.743834,0.421291,0.404659,0.598880,0.578513,0.546830,0.714064,0.649785,0.727069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2592,0.380453,0.667386,0.195877,0.647204,0.578513,0.526382,0.642732,0.665110,0.590431,0.680285,...,0.592637,0.670162,0.491237,0.297644,0.526146,1.000000,0.481141,0.623890,0.417242,0.682265
2593,0.260374,0.734768,0.221872,0.757787,0.546830,0.508879,0.722846,0.763350,0.762448,0.676956,...,0.640940,0.755850,0.344312,0.380152,0.713331,0.481141,1.000000,0.653851,0.502982,0.720326
2594,0.384227,0.824508,0.314224,0.857116,0.714064,0.675510,0.819730,0.840162,0.793665,0.827908,...,0.730198,0.808838,0.432669,0.397289,0.698260,0.623890,0.653851,1.000000,0.627879,0.844489
2595,0.277409,0.553637,0.205434,0.650407,0.649785,0.754899,0.597336,0.640461,0.596703,0.622892,...,0.640303,0.634390,0.324305,0.288863,0.502550,0.417242,0.502982,0.627879,1.000000,0.632439


In [ ]:
negative_count = (data1.iloc[:, 1:].values < 0).sum()
print(f"음수 값 개수: {negative_count}")

음수 값 개수: 36


In [22]:
negative_count = (data2.iloc[:, 1:].values < 0).sum()
print(f"음수 값 개수: {negative_count}")

음수 값 개수: 0


In [23]:
negative_count = (data3.iloc[:, 1:].values < 0).sum()
print(f"음수 값 개수: {negative_count}")

음수 값 개수: 0


In [ ]:
def get_upper_triangle_sim(df, name):
    # mat = cosine_similarity(df.drop(columns='word').values)
    mat = df.drop(columns='word').values
    upper_tri_indices = np.triu_indices_from(mat, k=1)  # k=1 -> 대각선 제외
    upper_tri_values = mat[upper_tri_indices]
    
    print(f"[{name}]")
    print(f"  Min similarity: {np.min(upper_tri_values):.4f}")
    print(f"  Max similarity: {np.max(upper_tri_values):.4f}")
    print(f"  Mean similarity: {np.mean(upper_tri_values):.4f}")
    print(f"  Median similarity: {np.median(upper_tri_values):.4f}")
    print(f"  75% quantile: {np.percentile(upper_tri_values, 75):.4f}")
    
    return upper_tri_values

# --- 데이터 계산 부분 (생략) ---
sim_bert = get_upper_triangle_sim(data1, 'BERT word embedding')
sim_topic_bert = get_upper_triangle_sim(data2, 'BERT topic similarity')
sim_topic_lda = get_upper_triangle_sim(data3, 'LDA topic probability')

# --- 그리기 ---
plt.figure(figsize=(18, 5))

for i, (sim_values, title) in enumerate(zip(
    [sim_topic_lda, sim_bert, sim_topic_bert],
    ['LDA Word-Topic Probability', 'BERT Word Embedding', 'BERT Word-Topic Similarity']
)):
    plt.subplot(1, 3, i + 1)
    sns.histplot(sim_values, bins=50, kde=True)
    plt.title(f"{title} Cosine Similarity\n(Upper Triangle Only)")
    plt.xlabel("Similarity")
    plt.ylabel("Frequency")
    
    if i == 1:
        ax = plt.gca()
        ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))  # y축 정수 표시
        ax.ticklabel_format(axis='y', style='plain')                  # 지수 표기 끄기

plt.tight_layout()
plt.show()
